How is the pref score changing with noisy data (or smoother data)

In [ ]:
import math
import pandas as pd
import altair as alt
import numpy as np
import flammkuchen as fl

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.compute_cea2034 import estimated_inroom
from spinorama.compute_scores import scores
from spinorama.graph import graph_params_default, graph_freq, graph_spinorama
from spinorama.load import filter_graphs, load_normalize

df_all = fl.load('../cache.parse_all_speakers.h5')

Compare Klippel graphs with computed ones.

In [ ]:
def noise(df, amp):
    sdf = df.copy()
    l = len(sdf.Freq.values)
    for c in sdf.columns:
        if c != 'Freq':
            sdf[c] += amp*(np.random.rand(l)-0.5)
    return sdf

def smooth(df, amp):
    sdf = df.copy()
    l = len(sdf.Freq.values)
    for c in sdf.columns:
        if c != 'Freq':
            sdf[c] = sdf[c].rolling(amp).mean()
    sdf.fillna(df)
    return sdf

In [ ]:
def eval_score_noise(df_all, speaker, amplitude):
    df_klippel = df_all[speaker]['ASR']['asr']
    spl_H = noise(df_klippel['SPL Horizontal_unmelted'], amplitude)
    spl_V = noise(df_klippel['SPL Vertical_unmelted'], amplitude)
    df_smoothed = load_normalize(filter_graphs('Adam S2V', spl_H, spl_V))
    return scores(df_smoothed)['pref_score']

def eval_score_smooth(df_all, speaker, amplitude):
    df_klippel = df_all[speaker]['ASR']['asr']
    spl_H = smooth(df_klippel['SPL Horizontal_unmelted'], amplitude)
    spl_V = smooth(df_klippel['SPL Vertical_unmelted'], amplitude)
    df_smoothed = load_normalize(filter_graphs('Adam S2V', spl_H, spl_V))
    return scores(df_smoothed)['pref_score']

In [ ]:
eval_score_smooth(df_all, 'Adam S2V', 3)

In [ ]:
range_noise = np.linspace(0, 3, 10)
res_noise = [(a, eval_score_noise(df_all, 'Adam S2V', a)) for a in range_noise]


In [ ]:
range_smooth = [2, 3, 4, 5, 6, 7, 8, 9, 10, 20]
res_smooth = [(a, eval_score_smooth(df_all, 'Adam S2V', a)) for a in range_smooth]

In [ ]:
for i, (a, n) in enumerate(res_noise):
    print('Noise +/-{:.1f}dB score {:.1f}'.format(a, n))

In [ ]:
for i, (a, n) in enumerate(res_smooth):
    print('Size of rolling window #{:2d} score {:.1f}'.format(a, n))